In [2]:
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

((x_train, y_train), (x_test, y_test)) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

Question 1 : A Convolutional Neural Network 
---

In this question, we ask you to build a model closely matching the above picture.

The model must have:

- **3 convolutional blocks** with a number of filters of respectively $16$, $32$, and $64$; **each block containing**:
    - A *Conv2D* layer with a *ReLU* activation.
    - A Max Pooling layer ($2,2$ pool size)
- A dense layer with $28$ hidden units.
- A dense output layer with a softmax activation.

You are free to play with the meta-parameters of each of these layers, as long as you respect this structure.

Implement you neural network in the variable *model*. Just define and compile the network, don't fit it on the training data (your submission will likely time out if you do!).

In [3]:
kernel_size = (8, 8)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, padding='same', activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Conv2D(32, kernel_size=kernel_size, padding='same', activation='relu'), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Conv2D(64, kernel_size=kernel_size, padding='same', activation='relu'), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    
], name='convolutional_net')

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999),
    metrics=["accuracy"]
)

In [4]:
def prepare_dataset(X, y, batch_size, shuffle=True, augment=True):
    augmented_args = dict(
        rotation_range = 8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        shear_range = 0.3,# shear angle in counter-clockwise direction in degrees  
        width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
        vertical_flip=True # randomly flip images
    
        # horizontal_flip=True,
        # vertical_flip=True,
        # rotation_range=20,
        # width_shift_range=0.2,
        # height_shift_range=0.2
        )
    args = {} if not augment else augmented_args
    
    ds = tf.keras.preprocessing.image.ImageDataGenerator(**args).flow(X, y, batch_size=batch_size, shuffle=shuffle)

    return ds

train_ds = prepare_dataset(x_train, y_train, 256, augment=False)

callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True)
    # tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
]

model.fit(
    train_ds,
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=callbacks
)

model.save(f"./models/{model.name}.model", save_format='h5')

Epoch 1/30
235/235 [==============================] - 14s 39ms/step - loss: 2.5910 - accuracy: 0.4890 - val_loss: 0.5249 - val_accuracy: 0.8241
Epoch 2/30
235/235 [==============================] - 3s 11ms/step - loss: 0.4294 - accuracy: 0.8515 - val_loss: 0.3933 - val_accuracy: 0.8642
Epoch 3/30
235/235 [==============================] - 2s 11ms/step - loss: 0.3447 - accuracy: 0.8778 - val_loss: 0.3533 - val_accuracy: 0.8734
Epoch 4/30
235/235 [==============================] - 2s 10ms/step - loss: 0.3059 - accuracy: 0.8907 - val_loss: 0.3269 - val_accuracy: 0.8856
Epoch 5/30
235/235 [==============================] - 2s 10ms/step - loss: 0.2722 - accuracy: 0.9034 - val_loss: 0.3463 - val_accuracy: 0.8851
Epoch 6/30
235/235 [==============================] - 2s 10ms/step - loss: 0.2465 - accuracy: 0.9118 - val_loss: 0.3285 - val_accuracy: 0.8856
Epoch 7/30
235/235 [==============================] - 2s 10ms/step - loss: 0.2282 - accuracy: 0.9175 - val_loss: 0.3283 - val_accuracy: 0.885

Question 3: Multiple choice 
---

Select all valid affirmations

- [ ] The number of filters of a convolutional layer does not influence its output shape.
- [ ] Reducing a lot the number of hidden units (e.g. down to 5 units) of the last layer before the output layer significantly decreases the achievable test accuracy (drop of 5% or more).
- [ ] Increasing or decreasing the number of convolutional blocks (yet, such number should be always >=1) of the network of Q1 and Q2 significantly influences the achievable test accuracy (drop or gain of 5% or more).
- [x] Increasing the pool size of a max pooling layer decreases the size of the output of that layer.
- [x] Using a pool size of (3,3) instead of (2,2) in the network of Q1 and Q2 produces 64 images of 1 pixel (at the output of the last convolutional block).
- [x] Increasing a lot the number of hidden units (e.g. up to 1000 units) of the last layer before the output layer significantly increases the achievable test accuracy (gain of 5% or more).
- [ ] Using a tanh activation function instead of relu significantly decreases (drop of 5% or more) the achievable test accuracy.
- [x] A high regularization on the weights of a layer increases overfitting.
- [ ] Having 64 images of 1 pixel at the output of the last convolutional block would prevent any learning.